In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-10 20:11:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-10-10 20:11:30 (16.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [ ]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for i in range(block_size):
  context=x[:i+1]
  target=y[i]
  print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [ ]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.382369041442871


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


In [ ]:
B,T,C= 4,8,32
head_size=16
x = torch.randn(B,T,C)
query=nn.Linear(C,head_size,bias=False)
key=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)
k=key(x)
q=query(x)
v=value(x)
score=q @ k.transpose(-2,-1)
tril=torch.tril(torch.ones(T,T))
print(tril)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


In [ ]:
score=score.masked_fill(tril==0,value=-float('inf'))
score/=(head_size**0.5)
score=F.softmax(score,dim=-1)
score[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5061, 0.4939, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3273, 0.3733, 0.2994, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1364, 0.3204, 0.1623, 0.3810, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1218, 0.1446, 0.2456, 0.0709, 0.4171, 0.0000, 0.0000, 0.0000],
        [0.1812, 0.1915, 0.0930, 0.1605, 0.1856, 0.1883, 0.0000, 0.0000],
        [0.0995, 0.1987, 0.0879, 0.1511, 0.1580, 0.0892, 0.2157, 0.0000],
        [0.1575, 0.1194, 0.0490, 0.1419, 0.1187, 0.0792, 0.1376, 0.1967]],
       grad_fn=<SelectBackward0>)

In [ ]:
out=score@v
out[0]

tensor([[-0.5117,  0.2885,  0.0498, -0.5087, -0.2714,  1.3020,  0.3679,  0.4241,
          0.6001,  1.2409, -0.4094,  0.4555, -1.3395, -0.4945, -0.6047, -0.6563],
        [-0.1963,  0.0238, -0.2538, -0.3739, -0.2184,  0.5120,  0.3519,  0.4364,
          0.2274,  1.1239, -0.4846,  0.1747, -0.6369, -0.3182, -0.4876, -0.3245],
        [-0.3040, -0.2670, -0.2259, -0.1236, -0.1279,  0.2032,  0.3055,  0.3193,
          0.1385,  0.7343, -0.4269, -0.1843, -0.4002, -0.2982, -0.2394, -0.2339],
        [ 0.0069, -0.7011, -0.1493, -0.2925, -0.0543, -0.0161,  0.5752,  0.1840,
         -0.0073,  0.9673, -0.4353,  0.0872, -0.1067,  0.0762,  0.0161,  0.0244],
        [ 0.1840, -0.1115, -0.9512, -0.2896,  0.0188,  0.3155, -0.2881,  0.0449,
         -0.0036,  0.1586, -0.1057, -0.1209, -0.0030, -0.4201, -0.1011, -0.4093],
        [ 0.1292,  0.0071, -0.5079, -0.3557, -0.0153,  0.2992, -0.0739,  0.1026,
          0.0943,  0.3900, -0.0504, -0.0623, -0.0107, -0.5361, -0.0244, -0.2887],
        [-0.0775, -0.1

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1)*head_size**-0.5
print(wei.var())
print(k.var())
print(q.var())

tensor(0.8790)
tensor(0.8717)
tensor(0.9883)


In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  def __call__(self, x):
    xmean=x.mean(1,keepdim=True)
    xvar=x.var(1,keepdim=True)
    xhat=(x - xmean) / torch.sqrt(xvar + self.eps)
    out= xhat*self.gamma+self.beta
    return out
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16
block_size = 32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
  def __init__(self,n_embd,head_size):
    super().__init__()
    self.query=nn.Linear(n_embd,head_size,bias=False)
    self.key=nn.Linear(n_embd,head_size,bias=False)
    self.value=nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))

  def forward(self,x):
    B,T,C= x.shape
    q=self.query(x)
    k=self.key(x)
    v=self.value(x)
    score=q@k.transpose(-2,-1) * C**-0.5
    score=score.masked_fill(self.tril[:T,:T]==0, value=-float('inf'))
    logits=F.softmax(score,dim=-1)
    out=logits@v
    return out

class MultiHeadAtt(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size=n_embd//n_head
    self.ma=nn.ModuleList([Head(n_embd,head_size) for _ in range(n_head)])
    self.proj=nn.Linear(n_embd,n_embd)

  def forward(self,x):
    out = torch.cat([f(x) for f in self.ma],dim=-1)
    pro = self.proj(out)
    return pro


class FeedForward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd,n_embd*4),
        nn.ReLU(),
        nn.Linear(4*n_embd,n_embd)
    )
  def forward(self,x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    self.at=MultiHeadAtt(n_embd,n_head)
    self.ffn=FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)

  def forward(self,x):
    x=x+self.at(self.ln1(x))
    x=x+self.ffn(self.ln2(x))
    return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.tok_emb=nn.Embedding(vocab_size,n_embd)
    self.pos_emb=nn.Embedding(block_size,n_embd)
    self.blocks=nn.Sequential(*[Block(n_embd,n_head) for _ in range(n_layer)])
    self.lin=nn.Linear(n_embd,vocab_size)
    self.ln_final=nn.LayerNorm(n_embd)

  def forward(self,idx,targets=None):
    B,T = idx.shape
    tok_em=self.tok_emb(idx)
    pos_em=self.pos_emb(torch.arange(T,device=device))
    x=pos_em+tok_em
    x=self.blocks(x)
    x=self.ln_final(x)
    logits = self.lin(x)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits=logits.view(B*T,C)
      targets=targets.view(B*T)

      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for i in range(max_new_tokens):
      idx_cond=idx[:,-block_size:]
      logits,loss=self(idx_cond)
      logits=logits[:,-1,:]
      probs=F.softmax(logits,dim=-1)
      next_token = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, next_token), dim=1)
    return idx


model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()), 'parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


209729 parameters
step 0: train loss 4.4111, val loss 4.4018
step 100: train loss 2.6551, val loss 2.6645
step 200: train loss 2.5082, val loss 2.5035
step 300: train loss 2.4155, val loss 2.4295
step 400: train loss 2.3477, val loss 2.3537
step 500: train loss 2.2967, val loss 2.3117
step 600: train loss 2.2417, val loss 2.2507
step 700: train loss 2.2016, val loss 2.2157
step 800: train loss 2.1639, val loss 2.1861
step 900: train loss 2.1248, val loss 2.1505
step 1000: train loss 2.0995, val loss 2.1252
step 1100: train loss 2.0694, val loss 2.1148
step 1200: train loss 2.0349, val loss 2.0754
step 1300: train loss 2.0262, val loss 2.0631
step 1400: train loss 1.9900, val loss 2.0352
step 1500: train loss 1.9665, val loss 2.0265
step 1600: train loss 1.9503, val loss 2.0312
step 1700: train loss 1.9437, val loss 2.0102
step 1800: train loss 1.9108, val loss 1.9959
step 1900: train loss 1.9038, val loss 1.9769
step 2000: train loss 1.8822, val loss 1.9843
step 2100: train loss 1.8722